In [26]:
# Import packages
import json
import pandas as pd
import plotly.express as px
import numpy as np

In [27]:
with open('data_extended_streaming\Streaming_History_Audio_2022-2024_1.json') as f1:
    data1 = json.load(f1)
with open('data_extended_streaming\Streaming_History_Audio_2018-2022_0.json') as f2:
    data2 = json.load(f2)

In [28]:
df1 = pd.DataFrame.from_dict(data1, orient='columns')
df2 = pd.DataFrame.from_dict(data2, orient='columns')

In [29]:
df = pd.concat([df1, df2], axis=0) 

In [30]:
df = df.drop(['ip_addr','spotify_track_uri','platform','offline_timestamp','incognito_mode','spotify_episode_uri'], axis=1)

In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 26496 entries, 0 to 18716
Data columns (total 13 columns):
 #   Column                             Non-Null Count  Dtype 
---  ------                             --------------  ----- 
 0   ts                                 26496 non-null  object
 1   ms_played                          26496 non-null  int64 
 2   conn_country                       26496 non-null  object
 3   master_metadata_track_name         26061 non-null  object
 4   master_metadata_album_artist_name  26061 non-null  object
 5   master_metadata_album_album_name   26061 non-null  object
 6   episode_name                       434 non-null    object
 7   episode_show_name                  434 non-null    object
 8   reason_start                       26496 non-null  object
 9   reason_end                         26496 non-null  object
 10  shuffle                            26496 non-null  bool  
 11  skipped                            26496 non-null  bool  
 12  offline  

In [32]:
df['ts'] = pd.to_datetime(df['ts'])

In [33]:
df['date'] = pd.to_datetime(df['ts']).dt.date

In [34]:
df['time'] = pd.to_datetime(df['ts']).dt.time

In [35]:
df['year'] = pd.to_datetime(df['ts']).dt.year

In [36]:
df.sort_values(by=['date'], inplace=True)
df.head()

,ts,ms_played,conn_country,master_metadata_track_name,master_metadata_album_artist_name,master_metadata_album_album_name,episode_name,episode_show_name,reason_start,reason_end,shuffle,skipped,offline,date,time,year
0,2018-10-19 02:04:12+00:00,3050,US,Wagon Wheel,Darius Rucker,True Believers,None,None,playbtn,fwdbtn,True,False,False,2018-10-19,02:04:12,2018
14,2018-10-19 02:35:49+00:00,231386,US,Puzzle Pieces,Saint Motel,Voyeur,None,None,fwdbtn,trackdone,True,False,False,2018-10-19,02:35:49,2018
13,2018-10-19 02:30:59+00:00,20498,US,Let It Snow! Let It Snow! Let It Snow! (with T...,Frank Sinatra,Christmas Songs By Sinatra,None,None,trackdone,fwdbtn,True,False,False,2018-10-19,02:30:59,2018
12,2018-10-19 02:30:37+00:00,335066,US,Snow (Hey Oh),Red Hot Chili Peppers,Stadium Arcadium,None,None,trackdone,trackdone,True,False,False,2018-10-19,02:30:37,2018
11,2018-10-19 02:25:01+00:00,267200,US,Say It (feat. Tove Lo) [Illenium Remix],Flume,Skin: The Remixes,None,None,fwdbtn,trackdone,True,False,False,2018-10-19,02:25:01,2018


In [48]:
artists = df.groupby(["year","master_metadata_album_artist_name"]).sum('ms_played')

In [50]:
artists.reset_index(drop=False, inplace=True)

In [52]:
artists.loc[artists['ms_played'].idxmax()]

year                                       2022
master_metadata_album_artist_name    Noah Kahan
ms_played                             246198805
shuffle                                    1364
skipped                                     159
offline                                     973
Name: 1187, dtype: object

In [62]:
artists['s_played'] = artists["ms_played"]/1000

In [64]:
noah = artists[artists["master_metadata_album_artist_name"]=="Noah Kahan"]

In [65]:
noah

,year,master_metadata_album_artist_name,ms_played,shuffle,skipped,offline,s_played
283,2019,Noah Kahan,59830734,249,0,40,59830.734
602,2020,Noah Kahan,10379298,92,0,39,10379.298
909,2021,Noah Kahan,126190027,571,0,544,126190.027
1187,2022,Noah Kahan,246198805,1364,159,973,246198.805
1570,2023,Noah Kahan,120205710,460,282,9,120205.710
1862,2024,Noah Kahan,36935993,6,50,0,36935.993


In [67]:
fig = px.line(noah, 
            x='year', 
            y='s_played', 
            labels={"year": 'Year', "s_played": 'Seconds Played'},
            #color='Working Day',
            #color_discrete_sequence=['darkcyan', 'rebeccapurple'],
            title='Seconds Played of Noah Kahan Over Time, 2019-2024')
fig.update_traces(mode="markers+lines", hovertemplate=None)
fig.update_layout(hovermode="x unified")